In [ ]:
pip install requests pandas
import pandas as pd
from itertools import combinations

In [ ]:
df = pd.read_csv("file.csv", on_bad_lines="skip")


In [ ]:
filtered_df = df[df['authorships.countries'].str.contains('US')]


In [ ]:
important_cols = [
    'id', 'type', 'type_crossref', 'cited_by_count', 'referenced_works_count',
    'publication_year', 'authorships.author.id', 'authorships.countries'
]

filtered_df = filtered_df[important_cols].copy()  # Ensuring a copy to avoid modifying the original DataFrame


In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

def get_author_country(openalex_id):
    """Fetch country code for a given author ID from OpenAlex API."""
    url = f"https://api.openalex.org/authors/{openalex_id}"
    try:
        response = requests.get(url)
        data = response.json()
        if 'affiliations' in data and data['affiliations']:
            return data['affiliations'][0]['institution'].get('country_code', 'Unknown')
    except Exception as e:
        print(f"Error fetching {openalex_id}: {e}")
    return "Unknown"

def fill_missing_countries(row):
    """Identify and fill missing country codes for a row in DataFrame."""
    author_ids = str(row['authorships.author.id']).split("|")  # Convert NaN to empty string
    country_codes = str(row['authorships.countries']).split("|")

    # Ensure both lists are the same length
    while len(country_codes) < len(author_ids):
        country_codes.append("")

    for i in range(len(author_ids)):
        if not country_codes[i]:  # Check for empty or None
            openalex_id = author_ids[i].split("/")[-1]  # Extract author ID
            country_codes[i] = get_author_country(openalex_id) or "Unknown"  # Ensure it's a string

    return "|".join(map(str, country_codes))  # Convert all items to string before joining


# Fill NaN values with empty string
filtered_df = filtered_df.fillna("")

# Apply function with progress bar
tqdm.pandas()
filtered_df["updated_authorships.countries"] = filtered_df.progress_apply(fill_missing_countries, axis=1)



In [ ]:
from itertools import permutations
import pandas as pd

# Function to process author pairs (now includes backward pairs)
def create_author_pairs(row):
    if pd.isna(row['authorships.author.id']):  # Handle missing values
        return [(None, None, None, None)]

    authors = [a.replace("https://openalex.org/", "") for a in row['authorships.author.id'].split("|")]
    countries = row['updated_authorships.countries'].split("|") if pd.notna(row['updated_authorships.countries']) else ["Unknown"] * len(authors)

    # If only one author, return (Author1, Country1, None, None)
    if len(authors) == 1:
        return [(authors[0], countries[0], None, None)]

    # Generate all ordered author pairs (including backward pairs)
    author_pairs = list(permutations(zip(authors, countries), 2))

    # Format output
    return [(a1[0], a1[1], a2[0], a2[1]) for a1, a2 in author_pairs]

# Apply function and explode into multiple rows
filtered_df['author_pairs'] = filtered_df.apply(create_author_pairs, axis=1)
filtered_df = filtered_df.explode('author_pairs')

# Convert tuple values into separate columns
filtered_df[['Author ID1', 'Author ID1 Country', 'Author ID2', 'Author ID2 Country']] = pd.DataFrame(filtered_df['author_pairs'].tolist(), index=filtered_df.index)

# Drop unnecessary columns
filtered_df = filtered_df.drop(columns=['author_pairs', 'authorships.author.id', 'updated_authorships.countries'])

# Reset index
filtered_df = filtered_df.reset_index(drop=True)

# Display result
print(filtered_df)


In [ ]:
final_df = filtered_df[filtered_df["Author ID1 Country"] == "US"]


In [ ]:
final_df.to_excel("final.xlsx", index=False)


### EXTRACTING PROFILES FROM TWITTER FOR AUTHOR ID-1


In [ ]:
# spliting the authorid from the link

author_tweetid = pd.read_csv(r'Input\authors_tweeters_2024_02.csv')
print(f'The shape of the input data: {author_tweetid.shape}')


author_tweetid['author_id_extracted'] = author_tweetid['author_id'].str.extract(r'https://openalex.org/(.+)')
author_tweetid['tweeter_id']= author_tweetid['tweeter_id'].astype(str)
print(f'The shape of the output data: {author_tweetid.shape}')
author_tweetid.to_csv("authors_tweeters_updated_file.csv", index=False)

In [ ]:
# Merging
import pandas as pd

# Load the Excel file into a DataFrame
left_file_path = r'merged_file.csv' # Replace with your actual file path
left_df = pd.read_csv(left_file_path, dtype= str)
# left_df = left_df.dropna(subset=['Author ID1'])
# left_df = left_df.drop_duplicates(subset=['Author ID1'])
# left_df['tweeter_id'] = left_df['tweeter_id'].astype(str)
print(f'The shape of left dataframe: {left_df.shape}')

right_filepath = r'Output_merged_full.xlsx'
right_df = pd.read_excel(right_filepath, dtype= str)
# right_df['User ID'] = right_df['User ID'].astype(str)

left_df_merged = left_df
left_df_merged =  left_df_merged.merge(right_df, left_on="tweeter_id", right_on="User ID", how="left",suffixes=('', '_right'), indicator=True)
left_df_merged['Is_Merged'] = left_df_merged['_merge'] == 'both'


columns_to_drop = [col for col in left_df_merged.columns if col.endswith('_right')]
left_df_merged = left_df_merged.drop(columns=columns_to_drop)
left_df_merged['tweeter_id'] = left_df_merged['tweeter_id'].astype(str)
left_df_merged['User ID']= left_df_merged['User ID'].astype(str)
left_df_merged.to_excel(r'Final_Merged.xlsx', index=False)
print(f'The shape of the output file: {left_df_merged.shape}')

In [ ]:
import tweepy
import pandas as pd
from tqdm import tqdm
import time
import config
from datetime import datetime

# Initialize the Tweepy Client
client = tweepy.Client(bearer_token=config.Bearer_token)

# Load the Excel file containing usernames
input_file_path = r'Twitter_Profile\Input_data_for_scraping_userprofiles\merged_file.xlsx'

# Save the cleaned data
cleaned_file_path = "Twitter_Profile\Output_Scraped_data\Cleaned_Twitter_Scraped_Data_Profiles_1000_New_test_3.xlsx"
twitter_handles_df = pd.read_excel(input_file_path).head(20)

# Clean the usernames: Remove leading/trailing whitespaces and drop blanks
# twitter_handles_df['username'] = twitter_handles_df['username'].str.strip()
twitter_handles_df = twitter_handles_df[twitter_handles_df['username'].notna()]

# Prepare a list to store scraped data
scraped_data = []

# Counter for partial file versions
partial_file_counter = 1

# Function to monitor rate limits
def check_rate_limit(response):
    remaining = int(response.headers.get('x-rate-limit-remaining', 1))
    reset_time = int(response.headers.get('x-rate-limit-reset', time.time() + 60))
    return remaining, reset_time

# Use tqdm for a progress bar
for username in tqdm(twitter_handles_df['username'], desc="Scraping Twitter Data"):
    # username = username.strip()  # Ensure no leading or trailing spaces
    while True:
        try:
            user = client.get_user(
                # username=username,
                id = username,
                user_fields=[
                    'id', 'name', 'username', 'created_at', 'description',
                    'public_metrics', 'location', 'entities', 'verified',
                    'profile_image_url', 'protected', 'profile_banner_url'
                ]
            )

            # Handle missing user data
            if user.data is None:
                print(f"[WARNING] No data found for username: {username}. Skipping.")
                scraped_data.append({
                    'User ID': username,
                    'Name': 'Not Found',
                    'Username': 'Not Found',
                    'Profile URL': f"https://twitter.com/i/user/{username}",
                    'Profile Image URL': 'Not Found',
                    'Profile Banner URL': 'Not Found',
                    'Protected Status': 'Not Found',
                    'Description': 'Not Found',
                    'Location': 'Not Found',
                    'Followers Count': 'Not Found',
                    'Following Count': 'Not Found',
                    'Tweet Count': 'Not Found',
                    'Media Count': 'Not Found',
                    'Listed Count': 'Not Found',
                    'Account Creation Date': 'Not Found',
                    'Verified Status': 'Not Found',
                    'External Link in Bio': 'Not Found'
                })
                break  # Skip to the next username

            external_link = 'N/A'
            if user.data.entities:
                if 'url' in user.data.entities and 'urls' in user.data.entities['url']:
                    urls = user.data.entities['url']['urls']
                    if len(urls) > 0 and 'expanded_url' in urls[0]:
                        external_link = urls[0]['expanded_url']

            # Construct profile URL
            profile_url = f"https://twitter.com/i/user/{user.data.id}"

            # Extract data
            profile_image_url = user.data.profile_image_url if hasattr(user.data, 'profile_image_url') else 'N/A'
            profile_banner_url = user.data.profile_banner_url if hasattr(user.data, 'profile_banner_url') else 'N/A'
            listed_count = user.data.public_metrics.get('listed_count', 'N/A')
            media_count = user.data.public_metrics.get('media_count', 'N/A')  # line to retrieve media count
            protected_status = user.data.protected

            scraped_data.append({
                'User ID': user.data.id,
                'Name': user.data.name,
                'Username': user.data.username,
                'Profile URL': profile_url,
                'Profile Image URL': profile_image_url,
                'Profile Banner URL': profile_banner_url,
                'Protected Status': protected_status,
                'Description': user.data.description,
                'Location': user.data.location if user.data.location else 'N/A',
                'Followers Count': user.data.public_metrics['followers_count'],
                'Following Count': user.data.public_metrics['following_count'],
                'Tweet Count': user.data.public_metrics['tweet_count'],
                'Media Count': media_count,  # Include the media count in the output
                'Listed Count': listed_count,
                'Account Creation Date': user.data.created_at,
                'Verified Status': user.data.verified,
                'External Link in Bio': external_link})

            break  # Exit the while loop on successful fetch
        except tweepy.TooManyRequests as e:
            # Save progress to a local CSV file with incrementing filename
            partial_output_file = f'Partial_Scraped_data\Twitter_Scraped_Data_Partial_german_users{partial_file_counter}.csv'
            pd.DataFrame(scraped_data).to_csv(partial_output_file, index=False, encoding='utf-8-sig')
            print(f"[PROGRESS SAVED] Partial data saved locally to {partial_output_file}")
            partial_file_counter = partial_file_counter + 1  # Increment the counter for the next file

            # Handle rate limit exception
            remaining, reset_time = check_rate_limit(e.response)
            sleep_time = max(0, reset_time - time.time())
            print(f"[PAUSE] Rate limit reached. Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
            print("[RESUME] Resuming data scraping...")
        except tweepy.TweepyException as e:
            # Log other errors and continue with the next username
            print(f"[ERROR] Failed to fetch data for username: {username}. Error: {e}")
            scraped_data.append({
                'User ID': username,
                'Name': 'Error',
                'Username': 'Error',
                'Profile URL': f"https://twitter.com/i/user/{username}",
                'Profile Image URL': 'Error',
                'Profile Banner URL': 'Error',
                'Protected Status': 'Error',
                'Description': 'Error',
                'Location': 'Error',
                'Followers Count': 'Error',
                'Following Count': 'Error',
                'Tweet Count': 'Error',
                'Media Count': 'Error',  # Add media count to error case
                'Listed Count': 'Error',
                'Account Creation Date': 'Error',
                'Verified Status': 'Error',
                'External Link in Bio': str(e)
            })
            break  # Skip to the next username on other errors

# Convert the list of scraped data to a DataFrame
scraped_data_df = pd.DataFrame(scraped_data)
scraped_data_df['Account Creation Date'] = pd.to_datetime(scraped_data_df['Account Creation Date'], errors='coerce')
scraped_data_df['Account Creation Date'] = scraped_data_df['Account Creation Date'].dt.strftime('%a %b %d %H:%M:%S %z %Y')
# Save the DataFrame to a final CSV file
output_file_path = 'Partial_Scraped_data\Twitter_Scraped_Data_userprofiles.csv'
scraped_data_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')
print(f"Final scraped data has been saved locally to {output_file_path}")

# Cleaning the data
# Load the data
data = pd.read_csv(output_file_path)

# Function to clean string columns by stripping leading/trailing whitespace and collapsing multiple spaces
def clean_text(text):
    if pd.isna(text):
        return text
    else:
        # Strip leading/trailing whitespace and replace multiple spaces with a single space
        return ' '.join(text.strip().split())

# Apply the cleaning function to all string columns in the dataframe
string_columns = data.select_dtypes(include=['object']).columns
data_cleaned = data.copy()
for column in string_columns:
    data_cleaned[column] = data[column].apply(clean_text)

data_cleaned["User ID"] = data_cleaned['User ID'].astype(str)

data_cleaned.to_excel(cleaned_file_path, index=False)

print(f"Final scraped data has been cleaned and saved locally to {cleaned_file_path}")
